## Modelando o Sistema Bancário em POO com Python

In [4]:
from abc import ABC, abstractclassmethod, abstractproperty
from datetime import datetime

1 - Classe pai <mark>Cliente</mark> recebe o atributo <i>Endereço</i>.

In [5]:
class Cliente:
    def __init__(self, endereco):
        self.endereco = endereco
        self.contas = []

    def realizar_transacao(self, conta, transacao):
        transacao.registrar(conta)

    def adicionar_conta(self, conta):
        self.contas.append(conta)

 - A classe <mark>Pessoa Física</mark> herda <i>Endereço</i> da classe <mark>Cliente</mark> mais os atributos <i>Nome, CPF e Data de Nascimento.</i>

In [6]:
class PessoaFisica(Cliente):
    def __init__(self, nome, cpf, data_nascimento, endereco):
        super().__init__(endereco)
        self.nome = nome
        self.cpf = cpf
        self.data_nascimento = data_nascimento

 2 - A classe <mark>Conta</mark> recebe os atributos <i>Cliente, Agência, Número, Saldo e Histórico.</i>

In [7]:
class Conta:
    def __init__(self, numero, cliente):
        self._cliente = cliente
        self._agencia = "0001"
        self._numero = numero
        self._saldo = 0
        self._historico = Historico()
        
        # Mapeando ClassMethod de nova conta e as propriedades dos atributos privados
        @classmethod
        def nova_conta(cls, cliente, numero):
            return cls(numero, cliente)

        @property
        def saldo(self):
            return self._saldo

        @property
        def numero(self):
            return self._numero

        @property
        def agencia(self):
            return self._agencia

        @property
        def cliente(self):
            return self._cliente

        @property
        def historico(self):
            return self._historico
        
        
        # Definindo as operações Saque e Depósito
        def sacar(self, valor):
            saldo = self.saldo
            excedeu_saldo = valor > saldo # verifica valor é maior que saldo disponível

            if excedeu_saldo:
                print("\n Operação não concluida. Saldo insuficiente.")

            elif valor > 0: # se o valor não for maior que saldo e for válido: debíta valor de saldo
                self._saldo -= valor 
                print("\n Saque realizado com sucesso!")
                return True

            else: # se for valor inválido (valor negativo)
                print("\n Operação não concluida. Valor inválido.")

            return False

        def depositar(self, valor):
            if valor > 0: 
                self._saldo += valor # se valor for maior que zero: credíta valor em saldo
                print("\n Depósito realizado com sucesso!")
            else:
                print("\n Operação não concluida. Valor inválido.")
                return False

            return True

 - A classe <mark>Conta Corrente</mark> herda de <mark>Conta</mark> mais os atributos <i>Limite e Limite de Saques.</i>

In [10]:
class ContaCorrente(Conta):
    def __init__(self, numero, cliente, limite=500, limite_saques=3):
        super().__init__(numero, cliente)
        self.limite = limite
        self.limite_saques = limite_saques

    def sacar(self, valor):
        # List Comprehension: Um "For loop" que percorre histórico de transações, verifica se o tipo é = a Saque e conta o número de transações através do tamanho da lista (len).
        numero_saques = len([transacao for transacao in self.historico.transacoes if transacao["tipo"] == "Saque"])

        # Verifica número de saques e limite de valor para saques.
        excedeu_limite = valor > self.limite
        excedeu_saques = numero_saques >= self.limite_saques

        if excedeu_limite:
            print("\n Operação não concluida. Valor excede o limite disponível.")

        elif excedeu_saques:
            print("\n Operação não concluida. Número máximo de saques excedido.")

        else:
            # Após a confirmação dos requisitos, para realizar a operação de saque chama a função sacar da classe pai "Conta".
            return super().sacar(valor)

        return False

    # Método "str" é a representação da classe Conta que vai apresentar os dados no extrato
    def __str__(self):
        return f"""\
            Agência:\t{self.agencia}
            C/C:\t\t{self.numero}
            Titular:\t{self.cliente.nome}
        """

3 - A Classe <mark>Histórico</mark> recebe cada transação realizada (saque ou depósito) em uma lista.

In [12]:
class Historico:
    def __init__(self):
        self._transacoes = []

    @property
    def transacoes(self):
        return self._transacoes

    # adiciona cada transação - em forma de dicionário - na lista de transações.
    def adicionar_transacao(self, transacao):
        self._transacoes.append(
            {
                "tipo": transacao.__class__.__name__,
                "valor": transacao.valor,
                "data": datetime.now().strftime("%d-%m-%Y %H:%M:%s"), # para obter a data atual do sistema formatada
            }
        )

4 - Classe abstrata <mark>Transação</mark> extende da classe utilitária <mark>ABC</mark>.

In [13]:
class Transacao(ABC):
    @property
    @abstractproperty
    def valor(self):
        pass

    @abstractclassmethod
    def registrar(self, conta):
        pass

5 - A Classe <mark>Saque</mark> extende da classe <mark>Transação</mark> mais o atributo <i>valor</i>.

In [14]:
class Saque(Transacao):
    def __init__(self, valor):
        self._valor = valor # armazena o valor do saque no atributo valor

    @property
    def valor(self):
        return self._valor

    def registrar(self, conta):
        sucesso_transacao = conta.sacar(self.valor)

        if sucesso_transacao:
            # se a transação for realizada, adiciona a tranzação no histórico
            conta.historico.adicionar_transacao(self)

6 - A Classe <mark>Depósito</mark> extende da classe <mark>Transação</mark> mais o atributo <i>valor</i>.

In [16]:
class Deposito(Transacao):
    def __init__(self, valor):
        self._valor = valor

    @property
    def valor(self):
        return self._valor

    def registrar(self, conta):
        sucesso_transacao = conta.depositar(self.valor)

        if sucesso_transacao:
            # se a transação for realizada, adiciona a tranzação no histórico
            conta.historico.adicionar_transacao(self)